# Design patterns with first-class functions

## Refactoring strategy

### Classic strategy

In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price*self.quantity
        
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        self.__total = None

    def total(self):
        if not self.__total:
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion:
            return self.promotion.discounted(self.total())
        return self.total()

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:2.f}>'
        return fmt.format(self.total(), self.due())
    

class PromotionStrategy(ABC):
    @abstractmethod
    def discount(price):
        pass

class FidelityPromotion(PromotionStrategy):

    @staticmethod
    def discounted(price):
        final_price = price * 0.90
        return final_price

class BulkItemPromotion(PromotionStrategy):

    @staticmethod
    def discounted(price):
        final_price = price * 0.85
        return final_price

        


gino = Customer('gino', 'high')
apple = LineItem('apple', 15, 2)
order = Order(gino, [apple], FidelityPromotion)
print(order.due())

27.0


### Function oriented strategy

In [9]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price*self.quantity
        
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        self.__total = None

    def total(self):
        if not self.__total:
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion:
            return self.promotion(self.total())
        return self.total()

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:2.f}>'
        return fmt.format(self.total(), self.due())
    

def fidelity_discounted(price):
    return price * 0.90

def bulk_discounted(price):
    return price * 0.85


gino = Customer('gino', 'high')
apple = LineItem('apple', 15, 2)
order = Order(gino, [apple], fidelity_discounted)
print(order.due())

27.0


In [16]:
# Suppose that in the above you want to automatically select the best promo

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price*self.quantity
        
class Order:
    def __init__(self, customer, cart, promotions=None):
        self.customer = customer
        self.cart = cart
        self.promotions = promotions
        self.__total = None

    def total(self):
        if not self.__total:
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotions:
            return min(promotion(self.total()) for promotion in self.promotions)
        return self.total()

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:2.f}>'
        return fmt.format(self.total(), self.due())
    

def fidelity_discounted(price):
    return price * 0.90

def bulk_discounted(price):
    return price * 0.85


gino = Customer('gino', 'high')
apple = LineItem('apple', 15, 2)
order = Order(gino, [apple], [fidelity_discounted, bulk_discounted])
print(order.due())

25.5


### Command design pattern

In [15]:
class MacroCommand:
    """A command that executes a list of commands"""

    def __init__(self,commands):
        self.commands = list(commands)

    def __call__(self):
        for command in self.commands:
            command()
